In [3]:
import sys
sys.path.append('/Users/eneminova/AsyncMethods')

from methods import MinibatchSGD, AsynchronousGD, RennalaSGD, WorkerState
from LinearRegression import linear_regression_loss, linear_regression_gradient
from LogisticRegression import logistic_regression_loss, logistic_regression_gradient
from MLP_mnist import SimpleNN, loss_from_vector, gradient_from_vector

import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from functools import partial
import scipy.stats as sps

from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
import scipy.stats as sps
import numpy as np
from scipy.special import expit
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.stats import ortho_group
from functools import partial
from collections import defaultdict
from IPython.display import clear_output
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'methods'

**LinReg on diabetes**

In [2]:
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

In [113]:
diabetes_X = np.hstack((diabetes_X, np.ones((len(diabetes_X), 1))))

In [115]:
n_workers = 4
time_distributions = [sps.norm(i+1, scale=1) for i in range(n_workers)]
batch_sizes = np.arange(1, n_workers + 1) * 10


n_iterations = 10000

w = np.zeros(diabetes_X.shape[1])

In [155]:
lr = 0.01

In [156]:
minibatch_sgd = MinibatchSGD(w, (diabetes_X, diabetes_y), time_distributions, linear_regression_loss, \
                              [partial(linear_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                  learning_rate=lr)
async_gd = AsynchronousGD(w, (diabetes_X, diabetes_y), time_distributions, linear_regression_loss, \
                              [partial(linear_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                  learning_rate=lr)
rennala_sgd = RennalaSGD(w, (diabetes_X, diabetes_y), time_distributions, linear_regression_loss, \
                              [partial(linear_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                  learning_rate=lr)
rennala_sgd.set_batch_size(5)

methods = {'Minibatch SGD' : minibatch_sgd,
           'Async GD' : async_gd,
           'Rennala SGD' : rennala_sgd}

In [157]:
times = dict()
losses = dict()
for method_name in methods:
    w = np.zeros(diabetes_X.shape[1])
    current_x, loss_history, computation_times, x_history = methods[method_name].run_steps(n_iterations)
    times[method_name] = computation_times
    losses[method_name] = loss_history

In [ ]:
plt.figure(figsize=(10, 6))
for method_name in methods:
    plt.plot(times[method_name], losses[method_name], label=method_name)

plt.xlabel('Time')
plt.ylabel('Loss')
plt.title('Optimization Methods Comparison, Linear Regression')
plt.legend()
plt.grid(True)
plt.show()

**LogReg on Mushroom**

In [ ]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("prishasawhney/mushroom-dataset")

data = pd.read_csv(f"{path}/mushroom_cleaned.csv")

In [186]:
scaler = MinMaxScaler()

x = data.drop('class', axis=1).values
x = scaler.fit_transform(x)
y = 2*data['class'].values - 1

In [199]:
data_mushroom = (x, y)

In [208]:
lr = 0.01
w = np.zeros(x.shape[1])

In [209]:
minibatch_sgd = MinibatchSGD(w, data_mushroom, time_distributions, logistic_regression_loss, \
                              [partial(logistic_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                  learning_rate=lr)
async_gd = AsynchronousGD(w, data_mushroom, time_distributions, logistic_regression_loss, \
                              [partial(logistic_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                  learning_rate=lr)
rennala_sgd = RennalaSGD(w, data_mushroom, time_distributions, logistic_regression_loss, \
                              [partial(logistic_regression_gradient, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                  learning_rate=lr)
rennala_sgd.set_batch_size(5)

methods = {'Minibatch SGD' : minibatch_sgd,
           'Async GD' : async_gd,
           'Rennala SGD' : rennala_sgd}

In [210]:
times = dict()
losses = dict()
for method_name in methods:
    w = np.zeros(diabetes_X.shape[1])
    current_x, loss_history, computation_times, x_history = methods[method_name].run_steps(1000)
    times[method_name] = computation_times
    losses[method_name] = loss_history

In [ ]:
plt.figure(figsize=(10, 6))
for method_name in methods:
    plt.plot(times[method_name], losses[method_name], label=method_name)

plt.xlabel('Time')
plt.ylabel('Loss')
plt.title('Optimization Methods Comparison, Logistic Regression on Mushroom')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
np.linalg.norm(logistic_regression_gradient(data_mushroom, x_history[-1]))

**NN on MNIST**

In [370]:
def load_mnist(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # Normalize to mean 0.5, std 0.5
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

    return train_dataset, test_dataset

In [371]:
mnist_train, mnist_test = load_mnist()

In [372]:
model = SimpleNN()

In [396]:
k = 10
subset_indices = torch.randperm(len(mnist_train))[:k]
subset_train = torch.utils.data.Subset(mnist_train, subset_indices)
train_loader = DataLoader(subset_train, batch_size=64, shuffle=True)


In [406]:
criterion = nn.CrossEntropyLoss()

In [411]:
num_param = sum(p.numel() for p in model.parameters())
num_param

12730

In [415]:
start = torch.rand((1, num_param))

In [416]:
minibatch_sgd = MinibatchSGD(start, mnist_train, time_distributions, partial(loss_from_vector, model=model, criterion=criterion), \
                              [partial(gradient_from_vector, model=model, criterion=criterion, batch_size=batch_sizes[i]) for i in range(n_workers)], \
                                  learning_rate=lr)

In [417]:
times = dict()
losses = dict()
w = np.zeros(diabetes_X.shape[1])
current_x, loss_history, computation_times, x_history = minibatch_sgd.run_steps(1)
times[method_name] = computation_times
losses[method_name] = loss_history